In [1]:
import requests
from bs4 import BeautifulSoup

### Naver Crawling Test
- 웹툰명(w_name)
- 해당 웹툰 링크

    - 소개글
    - 장르 : 쪼개져있는 장르를 모아야한다.
    - 작가
    - 완결여부
    - 평점
    - 하트수

In [21]:
web_dic = {}

web_dic['webtoon_name'] = []
web_dic['webtoon_link'] = []
web_dic['webtoon_finished'] = []

response = requests.get("https://comic.naver.com/webtoon/genre.nhn?genre=comic")
dom = BeautifulSoup(response.content, "html.parser")
webtoons = dom.select(".list_area > ul.img_list > li")
for webtoon in webtoons:
    web_dic['webtoon_name'].append(webtoon.select_one("dl > dt > a").get("title"))
    web_dic['webtoon_link'].append("https://comic.naver.com" + webtoon.select_one("dl > dt > a").get("href"))
#     print(len(webtoon.select("div > a > img")))
    if len(webtoon.select("div > a > img")) > 1:
#         print(webtoon.select("div > a > img")[1].get("alt"))
        web_dic['webtoon_finished'].append(1 if webtoon.select("div > a > img")[1].get("alt") == "완결" else 0)
    else:
        web_dic['webtoon_finished'].append(0)

In [ ]:
def naver_webtoon(genre):
    
    dic = {}
    dic['unique_id'] = []
    dic['webtoon_name'] = []
    dic['author'] = []
    dic['total_score'] = []
    dic['webtoon_link'] = []
    dic['image_link'] = []
    dic['webtoon_finished'] = []
    
    # first url for title
    # get title with full name
    url = "https://comic.naver.com/webtoon/genre.nhn?view=list&order=ViewCount&genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    webtoons = dom.select(".list_area > ul.img_list > li")
    for webtoon in webtoons:
        web_dic['webtoon_name'].append(webtoon.select_one("dl > dt > a").get("title"))
        dic['author'].append(webtoon.select_one("td:nth-of-type(3) > a").text)    
    
    # second url for the others
    url =  "https://comic.naver.com/webtoon/genre.nhn?genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".list_area > .img_list > li")


    # author, total score
    for keyword in keywords:
        dic['total_score'].append(keyword.select_one("dl > dd > .rating_type > strong").text)

    # unique id 
    unique_id_list = dom.select("dl > dt > a")
    for i in range(len(unique_id_list)):
        idx = unique_id_list[i].get('href').index("=")
        dic['unique_id'].append(unique_id_list[i].get('href')[idx+1:])

    # main link
    main_link_list = dom.select("dl > dt > a")
    for i in range(len(main_link_list)):
        dic['main_link'].append("https://comic.naver.com" + main_link_list[i].get('href'))

    # image link
    image_link_list = dom.select(".list_area > .img_list > li > .thumb > a > img:nth-of-type(1)")
    for i in range(len(image_link_list)):
        dic['image_link'].append(image_link_list[i].get('src'))
        
    df = pd.DataFrame(dic)
    df[genre] = genre
    df['author'] = df['author'].map(lambda x : "".join(x.split()))
    return df

In [23]:
web_dic['webtoon_link']

['https://comic.naver.com/webtoon/list.nhn?titleId=325629',
 'https://comic.naver.com/webtoon/list.nhn?titleId=570503',
 'https://comic.naver.com/webtoon/list.nhn?titleId=15640',
 'https://comic.naver.com/webtoon/list.nhn?titleId=55150',
 'https://comic.naver.com/webtoon/list.nhn?titleId=22052',
 'https://comic.naver.com/webtoon/list.nhn?titleId=651673',
 'https://comic.naver.com/webtoon/list.nhn?titleId=498158',
 'https://comic.naver.com/webtoon/list.nhn?titleId=680911',
 'https://comic.naver.com/webtoon/list.nhn?titleId=20853',
 'https://comic.naver.com/webtoon/list.nhn?titleId=434644',
 'https://comic.naver.com/webtoon/list.nhn?titleId=400737',
 'https://comic.naver.com/webtoon/list.nhn?titleId=15441',
 'https://comic.naver.com/webtoon/list.nhn?titleId=26316',
 'https://comic.naver.com/webtoon/list.nhn?titleId=409630',
 'https://comic.naver.com/webtoon/list.nhn?titleId=715772',
 'https://comic.naver.com/webtoon/list.nhn?titleId=103759',
 'https://comic.naver.com/webtoon/list.nhn?tit

In [10]:
web_dic['webtoon_finished']

[]

In [ ]:
def naver_webtoon(genre):
    
    dic = {}
    dic['unique_id'] = []
    dic['title'] = []
    dic['author'] = []
    dic['total_score'] = []
    dic['main_link'] = []
    dic['image_link'] = []
    dic['update_date'] = []
    
    # first url for title
    # get title with full name
    url = "https://comic.naver.com/webtoon/genre.nhn?view=list&order=ViewCount&genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select("#content > div.list_area.table_list_area > table > tbody > tr")
    for keyword in keywords:
        dic['title'].append(keyword.select_one("td > a > strong").text)
        dic['author'].append(keyword.select_one("td:nth-of-type(3) > a").text)    
    
    # second url for the others
    url =  "https://comic.naver.com/webtoon/genre.nhn?genre=" + genre
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".list_area > .img_list > li")


    # author, total score
    for keyword in keywords:
        dic['total_score'].append(keyword.select_one("dl > dd > .rating_type > strong").text)
        dic['update_date'].append(keyword.select_one("dl > dd.date2").text)

    # unique id 
    unique_id_list = dom.select("dl > dt > a")
    for i in range(len(unique_id_list)):
        idx = unique_id_list[i].get('href').index("=")
        dic['unique_id'].append(unique_id_list[i].get('href')[idx+1:])

    # main link
    main_link_list = dom.select("dl > dt > a")
    for i in range(len(main_link_list)):
        dic['main_link'].append("https://comic.naver.com" + main_link_list[i].get('href'))

    # image link
    image_link_list = dom.select(".list_area > .img_list > li > .thumb > a > img:nth-of-type(1)")
    for i in range(len(image_link_list)):
        dic['image_link'].append(image_link_list[i].get('src'))
        
    df = pd.DataFrame(dic)
    df[genre] = genre
    df['author'] = df['author'].map(lambda x : "".join(x.split()))
    return df